In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from cv2 import imread
from PIL import Image
from matplotlib.image import imread
from dash import Dash, html, dcc, Input, Output, State , callback_context
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import base64
from datetime import datetime
#
csvdmgp1 = pd.read_csv('esea_master_dmg_demos.part1.csv')
csvkillsp1 = pd.read_csv('esea_master_kills_demos.part1.csv')
csvdemosp1 = pd.read_csv('esea_meta_demos.part1.csv')
master_demos = pd.read_csv('mm_master_demos.csv', index_col=0)
map_data = pd.read_csv('map_data.csv', index_col=0)
print('Dados retirados')
# ScatterPlot + Data
def scatterPlotData(file,len_games,maps,allfiles):
    filtered_csvdemosp1_XGames = file[file['file'].isin(allfiles[:len_games])]
    filtered_csvdemosp1_XGames_XMaps = filtered_csvdemosp1_XGames[filtered_csvdemosp1_XGames['map'].isin(maps)]
    winner_val = []
    mean_val = []
    for i in range(len(filtered_csvdemosp1_XGames_XMaps)):
        if filtered_csvdemosp1_XGames_XMaps['winner_side'].iloc[i] == 'CounterTerrorist':
            winner_val.append(filtered_csvdemosp1_XGames_XMaps['ct_eq_val'].iloc[i])
        else:
            winner_val.append(filtered_csvdemosp1_XGames_XMaps['t_eq_val'].iloc[i])
        mean_val.append((filtered_csvdemosp1_XGames_XMaps['ct_eq_val'].iloc[i]+filtered_csvdemosp1_XGames_XMaps['t_eq_val'].iloc[i])/2)
    filtered_csvdemosp1_XGames_XMaps['winner_val'] = winner_val
    filtered_csvdemosp1_XGames_XMaps['mean_val'] = mean_val
    filtered_csvdemosp1_XGames_XMaps = filtered_csvdemosp1_XGames_XMaps.sort_values(by='round')
    df_medias_por_ronda = (
        filtered_csvdemosp1_XGames_XMaps
        .groupby('round')
        .agg({
            'ct_eq_val': 'mean',
            't_eq_val': 'mean',
            'winner_val': 'mean',
            'mean_val': 'mean',
            'winner_side': lambda x: x.value_counts().idxmax(),
            'round_type': lambda x: x.value_counts().idxmax()
        })
        .reset_index()
    )
    numeric_cols = ['ct_eq_val', 't_eq_val', 'winner_val', 'mean_val']
    df_medias_por_ronda[numeric_cols] = df_medias_por_ronda[numeric_cols].round(2)
    return df_medias_por_ronda
#
def dataLenMap(data,data_demos,x,maps,allfiles):
    data_demos = data_demos[data_demos['file'].isin(allfiles[:x])]
    data_demos = data_demos[data_demos['map'].isin(maps)]
    demos_unique = data_demos['file'].unique()
    filtered_csvdata_XGames_XMaps = data[data['file'].isin(demos_unique)]
    return filtered_csvdata_XGames_XMaps
# DADOS
allfilesdemosp1 = csvdemosp1['file'].unique()
allfiles = allfilesdemosp1
len_games = len(allfilesdemosp1)
maps = ['de_mirage','de_dust2','de_nuke','de_cache','de_inferno','de_overpass','de_train']
round_type = 'PISTOL_ROUND'
smap = 'de_mirage'
maps_mm = ['de_mirage', 'de_dust2', 'de_cbble','de_cache','de_inferno','de_overpass','de_train']
print("Filtragem de dados")
filtered_csvdmgp1 = dataLenMap(csvdmgp1,csvdemosp1,len_games,maps,allfiles)
filtered_killsp1 = dataLenMap(csvkillsp1,csvdemosp1,len_games,maps,allfiles)
filtered_csvdemosp1 = scatterPlotData(csvdemosp1,len_games,maps,allfiles)
filtered_csvdemosp1_2 = dataLenMap(csvdemosp1,csvdemosp1,len_games,maps,allfiles)
# Funções para os HeatMaps
def filterHeatMap(master_demos_aux,map_data,round_type,maps_mm,smap):
    master_demos_aux = master_demos_aux[master_demos_aux['map'].isin(maps_mm)]
    master_demos_aux = master_demos_aux.reset_index(drop=True)
    md = map_data.loc[master_demos_aux['map']]
    md[['att_pos_x', 'att_pos_y', 'vic_pos_x', 'vic_pos_y']] = (master_demos_aux.set_index('map')[['att_pos_x', 'att_pos_y', 'vic_pos_x', 'vic_pos_y']])
    md['att_pos_x'] = (md['ResX']*(md['att_pos_x']-md['StartX']))/(md['EndX']-md['StartX'])
    md['att_pos_y'] = (md['ResY']*(md['att_pos_y']-md['StartY']))/(md['EndY']-md['StartY'])
    md['vic_pos_x'] = (md['ResX']*(md['vic_pos_x']-md['StartX']))/(md['EndX']-md['StartX'])
    md['vic_pos_y'] = (md['ResY']*(md['vic_pos_y']-md['StartY']))/(md['EndY']-md['StartY'])

    master_demos_aux[['att_pos_x', 'att_pos_y', 'vic_pos_x', 'vic_pos_y']] = md[['att_pos_x', 'att_pos_y', 'vic_pos_x', 'vic_pos_y']].values

    rounds = list(range(1, 31))

    df_aux = master_demos_aux[(master_demos_aux['round'].isin(rounds)) & (master_demos_aux['round_type']==(round_type))]

    plot_df_t = df_aux.loc[(df_aux['map'] == smap) & (df_aux['att_side'] == 'Terrorist')]
    plot_df_ct = df_aux.loc[(df_aux['map'] == smap) & (df_aux['att_side'] == 'CounterTerrorist')]

    with open(fr"{smap}.png", "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode()
    bg_image_url = f"data:image/png;base64,{encoded_image}"

    return plot_df_t, plot_df_ct, bg_image_url
def createHeatMap(x, y, title,color,bg_image_url):
    return go.Figure(
        data=[
            go.Histogram2dContour(
                x=x,
                y=y,
                colorscale=color,
                contours=dict(showlines=False),
                ncontours=20,
                showscale=False,
                opacity=0.5
            ),
            go.Scatter(
                x=x, y=y,
                mode='markers',
                marker=dict(size=1, color='rgba(0,0,0,0)'),
                showlegend=False
            )
        ],
        layout=go.Layout(
            title=title,
            xaxis=dict(range=[0, 1024], showgrid=False, zeroline=False, visible=False),
            yaxis=dict(range=[0, 1024], showgrid=False, zeroline=False, visible=False, scaleanchor='x'),
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            images=[dict(
                source=bg_image_url,
                xref="x", yref="y",
                x=0, y=1024,
                sizex=1024, sizey=1024,
                sizing="stretch",
                layer="below"
            )],))
# Funções Vic por Type Round
def filterVicTypeRound(filtered_csvdemosp1):
    t_wins = filtered_csvdemosp1[filtered_csvdemosp1['winner_side'] == 'Terrorist']
    ct_wins = filtered_csvdemosp1[filtered_csvdemosp1['winner_side'] == 'CounterTerrorist']
    round_type_counts_t = t_wins['round_type'].value_counts(normalize=True) * 100
    round_type_counts_ct = ct_wins['round_type'].value_counts(normalize=True) * 100
    common_categories = list(set(round_type_counts_t.index) | set(round_type_counts_ct.index))
    round_type_counts_t = round_type_counts_t.reindex(common_categories, fill_value=0)
    round_type_counts_ct = round_type_counts_ct.reindex(common_categories, fill_value=0)
    index = np.arange(len(common_categories))
    return index, round_type_counts_t, round_type_counts_ct, common_categories
# Funções Vic por Per
def perVicdata(filtered_csvdmgp1):
    grouped_df = filtered_csvdmgp1['vic_side']
    counts = grouped_df.value_counts()
    total = counts.sum()
    ct_percent = (counts['CounterTerrorist'] / total) * 100
    t_percent = (counts['Terrorist'] / total) * 100
    return ct_percent, t_percent
# Funções Bomb planted
def bombdata(filtered_killsp1):
    last_actions = filtered_killsp1.groupby(['file', 'round']).tail(1)
    terrorist_last_actions = last_actions[last_actions['vic_side'] == 'Terrorist']
    bomb_counts = terrorist_last_actions['is_bomb_planted'].value_counts()
    total_actions = bomb_counts.sum()
    bomb_percentages = (bomb_counts / total_actions) * 100
    return bomb_percentages
# Funções hp e arm por wp
def wpdata(filtered_csvdmgp1):
    grouped_df_hp = filtered_csvdmgp1.groupby('wp')['hp_dmg'].sum().reset_index()
    grouped_df_hp = grouped_df_hp.sort_values(by='hp_dmg', ascending=False)
    total_hp_dmg = grouped_df_hp['hp_dmg'].sum()
    grouped_df_hp['percentagem_hp'] = (grouped_df_hp['hp_dmg'] / total_hp_dmg) * 100
    grouped_df_arm = filtered_csvdmgp1.groupby('wp')['arm_dmg'].sum().reset_index()
    grouped_df_arm = grouped_df_arm.sort_values(by='arm_dmg', ascending=False)
    total_arm_dmg = grouped_df_arm['arm_dmg'].sum()
    grouped_df_arm['percentagem_arm'] = (grouped_df_arm['arm_dmg'] / total_arm_dmg) * 100
    merged_df = pd.merge(grouped_df_arm, grouped_df_hp, on='wp', how='inner')
    merged_df['percentagem_total'] = merged_df['percentagem_arm'] + merged_df['percentagem_hp']
    filtered_df = merged_df[merged_df['percentagem_total'] >= 2]
    filtered_df = filtered_df.sort_values(by='percentagem_total', ascending=False)
    return filtered_df
#EXECUTE
print("EXECUTE")
index, round_type_counts_t, round_type_counts_ct, common_categories = filterVicTypeRound(filtered_csvdemosp1_2)
df_medias_por_ronda = scatterPlotData(csvdemosp1,len_games,maps,allfiles)
symbol_map = { 'PISTOL_ROUND': 'circle', 'ECO': 'cross', 'NORMAL': 'square'}
winner_map = {'Terrorist': 'Yellow','CounterTerrorist': 'Blue'}
df_medias_por_ronda['plotly_symbol'] = df_medias_por_ronda['round_type'].map(symbol_map)
df_medias_por_ronda['plotly_colors'] = df_medias_por_ronda['winner_side'].map(winner_map)
media_1_14 = df_medias_por_ronda[df_medias_por_ronda['round'].between(1, 14)]['mean_val'].mean()
media_15_29 = df_medias_por_ronda[df_medias_por_ronda['round'].between(15, 29)]['mean_val'].mean()
media_30_mais = df_medias_por_ronda[df_medias_por_ronda['round'] >= 30]['mean_val'].mean()
max_rounds = df_medias_por_ronda['round'].max()
ct_percent, t_percent = perVicdata(filtered_csvdmgp1)
bomb_percentages = bombdata(filtered_killsp1)
filtered_df = wpdata(filtered_csvdmgp1)
plot_df_t, plot_df_ct, bg_image_url = filterHeatMap(master_demos,map_data,round_type,maps_mm,smap)
allmaps = csvdemosp1['map'].unique().tolist()
alltyperounds = csvdemosp1['round_type'].unique().tolist()

#DASH
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?family=Inter:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]
app = Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

#dropdowns
app.css.append_css({
    'external_url': (
        '.Select-menu-outer {'
        '   background-color: #333 !important;'
        '   color: white !important;'
        '}'
        '.VirtualizedSelectOption {'
        '   background-color: #333 !important;'
        '}'
    )
})

app.layout = html.Div([
    # Cabeçalho com Logo
    html.Div([
        html.Img(src='https://i.ibb.co/5sZSZCY/logo.png',  # Substitua pelo seu URL de imagem
                 style={'height': '60px', 'margin-right': '20px'}),
        html.H1("CS2 Strategic Dashboard", 
                style={'color': 'white', 'margin': '0', 'fontFamily': 'Arial Black'})
    ], style={'background': '#1a1a1a', 'padding': '20px', 'display': 'flex', 'align-items': 'center'}),
    
    # Corpo Principal
    html.Div([
        # Sidebar de Filtros
        html.Div([
            html.H3('FILTROS', 
                    style={'color': 'white', 'borderBottom': '2px solid #444', 'paddingBottom': '10px'}),
            
            html.Label('Selecione o Mapa:', style={'color': 'white', 'marginTop': '15px'}),
            dcc.Dropdown(
                id='smap-filter',
                options=[{'label': x, 'value': x} for x in allmaps],
                placeholder="Todos os Mapas",
                style={'backgroundColor': '#333', 'color': 'white', 'border': '1px solid #555'}
            ),
            
            html.Label('Tipo de Rodada:', style={'color': 'white', 'marginTop': '15px'}),
            dcc.Dropdown(
                id='round-type-filter',
                options=[{'label': x, 'value': x} for x in alltyperounds],
                placeholder="Todos os Tipos",
                style={'backgroundColor': '#333', 'color': 'white', 'border': '1px solid #555'}
            ),
            
            html.Label('Número de Partidas:', style={'color': 'white', 'marginTop': '15px'}),
            dcc.Input(
                id='numero-input',
                type='number',
                min=1,
                max=8500,
                value=500,
                style={'backgroundColor': '#333', 'color': 'white', 'border': '1px solid #555'}
            ),
            
            html.Button(
                'Aplicar Filtros',
                id='print-button',
                style={
                    'background': '#FFA500',
                    'color': 'black',
                    'border': 'none',
                    'padding': '10px 20px',
                    'marginTop': '20px',
                    'fontWeight': 'bold'
                }
            )
        ], style={
            'background': '#2a2a2a',
            'padding': '20px',
            'width': '250px',
            'height': 'calc(100vh - 100px)',
            'position': 'fixed'
        }),
        
        # Gráficos
        html.Div([
            # Linha Superior
            html.Div([
                html.Div(
                    dcc.Graph(id="graph-2"),
                    style={'flex': '2', 'background': '#333', 'borderRadius': '10px', 'padding': '15px'}
                ),
                html.Div([
                    dcc.Graph(id="graph-3"),
                    dcc.Graph(id="graph-4")
                ], style={'flex': '1', 'display': 'flex', 'flexDirection': 'column', 'gap': '20px'})
            ], style={'display': 'flex', 'gap': '20px', 'marginBottom': '20px'}),
            
            # Linha do Meio
            html.Div([
                html.Div(
                    dcc.Graph(id="graph-1"),
                    style={'flex': '1', 'background': '#333', 'borderRadius': '10px', 'padding': '15px'}
                ),
                html.Div(
                    dcc.Graph(id='heatmap-ct'),
                    style={'flex': '1', 'background': '#333', 'borderRadius': '10px', 'padding': '15px'}
                )
            ], style={'display': 'flex', 'gap': '20px', 'marginBottom': '20px'}),
            
            # Linha Inferior
            html.Div([
                html.Div(
                    dcc.Graph(id="graph-5"),
                    style={'flex': '1', 'background': '#333', 'borderRadius': '10px', 'padding': '15px'}
                ),
                html.Div(
                    dcc.Graph(id='heatmap-t'),
                    style={'flex': '1', 'background': '#333', 'borderRadius': '10px', 'padding': '15px'}
                )
            ], style={'display': 'flex', 'gap': '20px'})
            
        ], style={
            'marginLeft': '270px',
            'padding': '20px',
            'background': '#1a1a1a',
            'minHeight': 'calc(100vh - 100px)'
        })
    ]),
    
    html.Div(id='dummy-output', style={'display': 'none'})
], style={'fontFamily': 'Arial, sans-serif'})

@app.callback(
    [Output("graph-1", "figure"),
     Output("graph-2", "figure"),
     Output('graph-3', 'figure'),
     Output('graph-4', 'figure'),
     Output('graph-5', 'figure')],
    [Input('smap-filter', 'value'),
     Input('round-type-filter', 'value'),
     Input('numero-input', 'value')]
)

def atualizar_graficos(mapa_selecionado, tipo_round_selecionado, num_partidas):
    filtered_data = filtrar_dados(
        mapa=mapa_selecionado,
        round_type=tipo_round_selecionado,
        num_partidas=num_partidas
    )

    fig1 = go.Figure()
    bar_width = 0.35

    fig1.add_trace(go.Bar(
        x=index - bar_width/2,
        y=round_type_counts_t,
        name='Terroristas',
        marker_color='yellow',
        width=bar_width,
        text=[f'{val:.1f}%' for val in round_type_counts_t],
        textposition='auto'
    ))

    fig1.add_trace(go.Bar(
        x=index + bar_width/2,
        y=round_type_counts_ct,
        name='Contra-Terroristas',
        marker_color='blue',
        width=bar_width,
        text=[f'{val:.1f}%' for val in round_type_counts_ct],
        textposition='auto'
    ))

    fig1.update_layout(
        title='Distribuição de Vitórias por Tipo de Round',
        xaxis=dict(
            title='Tipo de Round',
            tickvals=index,
            ticktext=common_categories,
            tickangle=-45
        ),
        yaxis=dict(title='Percentagem (%)', range=[0, 100]),
        barmode='group',
        plot_bgcolor='white',
        xaxis_autorange=True,
        yaxis_autorange=True
    )

    fig2 = go.Figure()

    fig2.add_trace(go.Scatter(
        x=df_medias_por_ronda['round'],
        y=df_medias_por_ronda['ct_eq_val'],
        mode='markers',
        name='CT Equip Value',
        marker=dict(
            color=df_medias_por_ronda['plotly_colors'],
            symbol=df_medias_por_ronda['plotly_symbol'],
            size=12,
            line=dict(width=1.5, color='blue')
        )
    ))

    fig2.add_trace(go.Scatter(
        x=df_medias_por_ronda['round'],
        y=df_medias_por_ronda['t_eq_val'],
        mode='markers',
        name='T Equip Value',
        marker=dict(
            color=df_medias_por_ronda['plotly_colors'],
            symbol=df_medias_por_ronda['plotly_symbol'],
            size=12,
            line=dict(width=1.5, color='yellow')
        )
    ))

    fig2.add_trace(go.Scatter(
        x=df_medias_por_ronda['round'],
        y=df_medias_por_ronda['mean_val'],
        mode='lines',
        name='Média de dinheiro gasto',
        line=dict(color='red', dash='dash')
    ))

    fig2.add_shape(
        type='line', x0=1, x1=14,
        y0=media_1_14, y1=media_1_14,
        line=dict(color='red', dash='dash')
    )
    fig2.add_shape(
        type='line', x0=15, x1=29,
        y0=media_15_29, y1=media_15_29,
        line=dict(color='red', dash='dash')
    )
    fig2.add_shape(
        type='line', x0=30, x1=max_rounds,
        y0=media_30_mais, y1=media_30_mais,
        line=dict(color='red', dash='dash')
    )

    fig2.add_vline(x=15, line=dict(color='black', dash='dash'),
                   annotation_text="Round 15", annotation_position="top left")
    fig2.add_vline(x=30, line=dict(color='black', dash='dash'),
                   annotation_text="Round 30", annotation_position="top left")

    fig2.update_layout(
        title='Scatter Plot: Round vs Equipamento Value (CT e T)',
        xaxis_title='Round',
        yaxis_title='Equipamento Value',
        legend_title='Lado Vencedor / Tipo de Round',
        template='plotly_white'
    )

    fig3 = go.Figure()

    fig3.add_trace(go.Bar(
        x=['Counter-Terrorist', 'Terrorist'],
        y=[ct_percent, t_percent],
        marker_color=['blue', 'yellow'],
        text=[f'{ct_percent:.1f}%', f'{t_percent:.1f}%'],
        textposition='auto'))

    fig3.update_layout(
        title='Percentagem de Vitórias por Equipa',
        xaxis_title='Equipas',
        yaxis_title='Percentagem (%)',
        yaxis=dict(range=[0, max(ct_percent, t_percent) + 5]),
        plot_bgcolor='white',
    )

    fig4 = go.Figure()

    fig4.add_trace(go.Bar(
        x=['Não Plantada', 'Plantada'],
        y=bomb_percentages.values,
        marker_color=['red', 'green'],
        text=[f'{val:.1f}%' for val in bomb_percentages.values],
        textposition='auto'
    ))

    fig4.update_layout(
        title='Percentagem de Bombas Plantadas<br>(Terrorist - Última Ação por Ronda)',
        xaxis_title='Status da Bomba',
        yaxis_title='Percentagem (%)',
        yaxis=dict(range=[0, max(bomb_percentages.values) + 5]),
        plot_bgcolor='white',
    )

    fig5 = go.Figure()

    fig5.add_trace(go.Bar(
        x=filtered_df['wp'],
        y=filtered_df['percentagem_arm'],
        name='Dano em Armadura',
        marker_color='darkred',
        text=[f'{val:.1f}%' for val in filtered_df['percentagem_arm']],
        textposition='inside',
        textfont=dict(size=15)
    ))

    fig5.add_trace(go.Bar(
        x=filtered_df['wp'],
        y=filtered_df['percentagem_hp'],
        name='Dano em Vida',
        marker_color='darkblue',
        text=[f'{val:.1f}%' for val in filtered_df['percentagem_hp']],
        textposition='inside',
        textfont=dict(size=15)
    ))

    fig5.update_layout(
        title='Percentagem de Dano por Arma (HP por cima de Armadura)',
        xaxis_title='Arma (wp)',
        yaxis_title='Percentagem do Dano Total (%)',
        barmode='stack',
        bargap=0.2,
        template='plotly_white',
        xaxis_tickangle=-45
    )


    return fig1,fig2,fig3,fig4,fig5

@app.callback(
    Output('dummy-output', 'children'),
    Input('print-button', 'n_clicks'),
    [State('smap-filter', 'value'),
     State('round-type-filter', 'value'),
     State('numero-input', 'value')],
    prevent_initial_call=True
)
def print_filters(n_clicks, mapa, tipo_round, num_partidas):
    if n_clicks:
        print(f"\n=== FILTROS ATIVOS ===")
        print(f"Mapa: {mapa if mapa else 'Não selecionado'}")
        print(f"Tipo de Round: {tipo_round if tipo_round else 'Não selecionado'}")
        print(f"Nº Partidas Analisadas: {num_partidas if num_partidas else 'Todas'}")
    
    return ''
def filtrar_dados(mapa, round_type, num_partidas):
    # filtered = []
    # if mapa:
    #     filtered = filtered[filtered['map'] == mapa]
    # if round_type:
    #     filtered = filtered[filtered['round_type'] == round_type]
    # if num_partidas:
    #     filtered = filtered.head(num_partidas)
    return None
@app.callback(
    Output('numero-input', 'value'),
    Input('numero-input', 'value')
)
def validate_input(value):
    if value < 1:
        return 1
    if value > 8500:
        return 8500
    return value

if __name__ == '__main__':
    print('RUNNING!')
    app.run(debug=True, port=8051)

Dados retirados
Filtragem de dados


C:\Users\User\AppData\Local\Temp\ipykernel_11656\2311507272.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_csvdemosp1_XGames_XMaps['winner_val'] = winner_val
C:\Users\User\AppData\Local\Temp\ipykernel_11656\2311507272.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_csvdemosp1_XGames_XMaps['mean_val'] = mean_val


EXECUTE


C:\Users\User\AppData\Local\Temp\ipykernel_11656\2311507272.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_csvdemosp1_XGames_XMaps['winner_val'] = winner_val
C:\Users\User\AppData\Local\Temp\ipykernel_11656\2311507272.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_csvdemosp1_XGames_XMaps['mean_val'] = mean_val


RUNNING!


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\resources.py:96: UserWarning:

You have set your config to `serve_locally=True` but A local version of .Select-menu-outer {   background-color: #333 !important;   color: white !important;}.VirtualizedSelectOption {   background-color: #333 !important;} is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\dash\resources.py:96: UserWarning:

You have set your config to `serve_locally=True` but A local version of .Select-menu-outer {   background-color: #333 !important;   color: white !important;}.VirtualizedSelectOption {   background-color: #333 !important;} is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources




=== FILTROS ATIVOS ===
Mapa: de_cache
Tipo de Round: ECO
Nº Partidas Analisadas: 500
